# Biblioteca para extrair o texto do PDF

In [ ]:
pip install PyPDF2

# Biblioteca que faz a tradução

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
!pip install gtts

# Áudio em espanhol

Ao clicar no botão para subir o PDF, ocorre a extração do texto, é dividido em páginas, traduz o texto e adiciona um botão para gerar áudio, processo é um pouco demorado

In [ ]:
import io
import logging
import re
import time
import os
from PyPDF2 import PdfReader
from googletrans import Translator
from gtts import gTTS
from ipywidgets import FileUpload, Button, VBox, Output
from IPython.display import display, HTML, Audio, clear_output

# Configuração do logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def translate_with_retry(text, translator, dest='pt', max_retries=3, delay=2):
    """Realiza a tradução de um texto com múltiplas tentativas em caso de falha."""
    if text is None:
        logging.warning("Texto fornecido para tradução é None.")
        return ""

    for i in range(max_retries):
        try:
            text = text.strip()
            if text:
                translation = translator.translate(text, dest=dest)
                if translation and translation.text:
                    return translation.text
                else:
                    logging.warning(f"Tradução falhou para a parte: {text}")
            else:
                logging.warning("Texto vazio fornecido para tradução.")
        except Exception as e:
            logging.error(f"Erro ao traduzir: {str(e)}")
        time.sleep(delay)
    return text

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens, por página."""
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        return reader.pages
    except Exception as e:
        logging.error(f"Erro ao extrair texto do PDF: {e}")
        return None

def format_text(text, page_number):
    """Formata o texto extraído para identificar títulos e subtítulos."""
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    formatted_text = re.sub(title_pattern, r'<h1 style="color: #800020;">\g<0></h1>', text)
    formatted_text = re.sub(subtitle_pattern, r'<h2 style="color: darkblue;">\g<0></h2>', formatted_text)

    return formatted_text, re.sub(r'<[^>]+>', '', text)  # Retorna texto formatado e texto limpo

def text_to_speech_gtts(text, lang='es'):
    """Converte texto para fala usando gTTS em espanhol."""
    try:
        audio_file_path = "temp_audio.mp3"
        tts = gTTS(text=text, lang=lang, slow=False)
        tts.save(audio_file_path)
        return audio_file_path
    except Exception as e:
        logging.error(f"Erro ao gerar áudio com gTTS: {e}")
        return None

def create_audio_player(audio_file_path):
    """Cria e exibe o player de áudio, depois remove o arquivo."""
    try:
        audio = Audio(audio_file_path, autoplay=False)
        display(audio)
        os.remove(audio_file_path)
        return audio
    except Exception as e:
        logging.error(f"Erro ao criar player de áudio: {e}")
        return None

def handle_pdf_upload(change):
    """Processa o upload do PDF, traduz o conteúdo e cria botões para gerar áudios."""
    try:
        if not change.new:
            display(HTML("Erro: Nenhum arquivo foi carregado."))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("Erro: O conteúdo do arquivo está vazio ou não foi carregado corretamente."))
            return

        pages = extract_text_from_pdf(content)

        if not pages:
            display(HTML("Erro: Não foi encontrado texto no PDF ou houve um problema na extração."))
            return

        translator = Translator()

        # Limpar a saída anterior
        clear_output(wait=True)
        display(upload_btn)

        # Processar página por página
        for page_number, page in enumerate(pages, start=1):
            page_text = page.extract_text()
            if page_text:
                formatted_text, clean_text = format_text(page_text, page_number)

                # Dividir o texto em partes menores para tradução
                text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
                translated_page = ""

                # Traduzir o texto em partes
                for i, part in enumerate(text_parts):
                    if part is not None:
                        try:
                            translated_part = translate_with_retry(part, translator)
                            translated_page += translated_part
                        except Exception as e:
                            logging.error(f"Erro ao traduzir parte {i+1} da página {page_number}: {e}")
                            translated_page += f"<p style='color: red;'>Erro na tradução da parte {i+1}: {part}</p>"
                    else:
                        translated_page += "<p style='color: red;'>Parte vazia ou não válida</p>"

                # Criar container para o áudio
                audio_output = Output()

                # Função para gerar áudio quando o botão for clicado
                def on_button_click(b, text=clean_text, output=audio_output):
                    with output:
                        output.clear_output()
                        audio_file = text_to_speech_gtts(text, lang='es')
                        if audio_file:
                            create_audio_player(audio_file)

                # Criar botão
                button = Button(description=f'Gerar Áudio - Página {page_number}',
                              style={'button_color': 'lightblue'})
                button.on_click(on_button_click)

                # Exibir seção
                display(HTML(f"""
                <h2 style="color: #800020;">Página {page_number}</h2>
                <div style="background-color: #f0f0f0; padding: 10px; margin-bottom: 10px; border-radius: 5px;">
                    <h3>Texto Original</h3>
                    <div style="margin-bottom: 15px;">{formatted_text}</div>
                    <h3>Texto Traduzido</h3>
                    <div>{translated_page}</div>
                </div>
                """))

                # Exibir botão e espaço para o áudio
                display(button)
                display(audio_output)
                display(HTML("<hr style='margin: 20px 0;'>"))  # Linha separadora

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo: {e}")
        display(HTML(f"Erro ao processar o arquivo: {str(e)}"))

# Instalar gTTS se necessário
try:
    from gtts import gTTS
except ImportError:
    !pip install gTTS
    from gtts import gTTS

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False, description='Carregar PDF')
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(HTML("<h1 style='color: #800020;'>Tradutor de PDF com Áudio</h1>"))
display(HTML("<p>Carregue um arquivo PDF para traduzir e gerar áudio em espanhol</p>"))
display(upload_btn)

# Áudio em inglês

Um das tentativas até dar certo, lê o PDF e gera o áudio com sotaque em **inglês** em textos de que não são de origem inglesa, porque é configurado com leitor em inglês, depois alterei a propriedade que era en(de inglês). Então serve para texto em inglês.



In [ ]:
import io
import logging
import re
import time
import os
from PyPDF2 import PdfReader
from googletrans import Translator
from gtts import gTTS
from ipywidgets import FileUpload, Button, HTML, VBox, IntProgress
from IPython.display import display, Audio, clear_output

# Configuração do logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def translate_with_retry(text, translator, dest='pt', max_retries=3, delay=2):
    """Realiza a tradução de um texto com múltiplas tentativas em caso de falha."""
    if text is None:
        logging.warning("Texto fornecido para tradução é None.")
        return ""

    for i in range(max_retries):
        try:
            text = text.strip()  # Remover espaços em branco no início e no fim
            if text:
                translation = translator.translate(text, dest=dest)
                if translation and translation.text:
                    return translation.text
                else:
                    logging.warning(f"Tradução falhou para a parte: {text}")
            else:
                logging.warning("Texto vazio fornecido para tradução.")
        except Exception as e:
            logging.error(f"Erro ao traduzir: {str(e)}")
        time.sleep(delay)
    return text

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens, por página."""
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        return reader.pages
    except Exception as e:
        logging.error(f"Erro ao extrair texto do PDF: {e}")
        return None

def format_text(text, page_number):
    """Formata o texto extraído para identificar títulos e subtítulos."""
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    formatted_text = re.sub(title_pattern, r'<h1 style="color: #800020;">\g<0></h1>', text)
    formatted_text = re.sub(subtitle_pattern, r'<h2 style="color: darkblue;">\g<0></h2>', formatted_text)

    return formatted_text

def text_to_speech_gtts(text, lang='en'):
    """Converte texto para fala usando gTTS e retorna o caminho do arquivo de áudio."""
    try:
        # Gerar o áudio usando gTTS e salvar em um arquivo temporário
        tts = gTTS(text, lang=lang)
        audio_file_path = "output_gtts.mp3"
        tts.save(audio_file_path)

        return audio_file_path
    except Exception as e:
        logging.error(f"Erro ao gerar áudio com gTTS: {e}")
        return None

def generate_audio(button, text, snippet):
    """Gera o áudio para o texto original em inglês e exibe o player com uma barra de progresso."""

    # Criar uma barra de progresso
    progress_bar = IntProgress(value=0, min=0, max=100, description='Gerando áudio:')
    display(progress_bar)

    try:
        # Limpar saída anterior (se necessário)
        clear_output(wait=True)
        display(progress_bar)

        # Gerar o áudio para o texto completo
        audio_file_path = text_to_speech_gtts(text, lang='en')  # Áudio apenas em inglês
        if audio_file_path:
            # Remover a barra de progresso após a conclusão
            progress_bar.value = 100
            time.sleep(1)  # Esperar um pouco para garantir que o progresso esteja completo
            clear_output(wait=True)  # Limpar saída anterior

            # Exibir o player de áudio
            display(Audio(audio_file_path, autoplay=False))  # Exibir áudio
            os.remove(audio_file_path)  # Limpar arquivo de áudio após exibir
        else:
            logging.error("Não foi possível gerar o áudio.")
            display(HTML("Erro ao gerar áudio."))

        # Reexibir o texto e os botões
        display(container)

    except Exception as e:
        # Remover a barra de progresso em caso de erro
        progress_bar.close()
        clear_output(wait=True)  # Limpar saída anterior
        logging.error(f"Erro ao gerar áudio: {e}")
        display(HTML(f"Erro ao gerar áudio: {e}"))

def handle_pdf_upload(change):
    """Processa o upload do PDF, exibe tradução e cria botões para gerar áudios do texto original."""
    try:
        if not change.new:
            display(HTML("Erro: Nenhum arquivo foi carregado."))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("Erro: O conteúdo do arquivo está vazio ou não foi carregado corretamente."))
            return

        pages = extract_text_from_pdf(content)

        if not pages:
            display(HTML("Erro: Não foi encontrado texto no PDF ou houve um problema na extração."))
            return

        translator = Translator()

        # Adicionar o botão de upload novamente
        clear_output(wait=True)  # Limpar saída anterior
        display(upload_btn)

        # Criar um container para manter o texto e os botões
        global container
        container = VBox()

        # Processar página por página
        for page_number, page in enumerate(pages, start=1):
            page_text = page.extract_text()
            if page_text:
                formatted_text = format_text(page_text, page_number)

                # Dividir o texto em partes menores para tradução
                text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
                translated_page = ""
                text_snippet = formatted_text[:500]  # Exibe os primeiros 500 caracteres do texto formatado

                # Traduzir o texto para o português
                for i, part in enumerate(text_parts):
                    if part is not None:
                        try:
                            translated_part = translate_with_retry(part, translator, dest='pt')
                            translated_page += translated_part
                        except Exception as e:
                            logging.error(f"Erro ao traduzir parte {i+1} da página {page_number}: {e}")
                            translated_page += f"<p style='color: red;'>Erro na tradução da parte {i+1}: {part}</p>"
                    else:
                        translated_page += "<p style='color: red;'>Parte vazia ou não válida</p>"

                # Adicionar texto original e traduzido ao container
                page_display = HTML(f"""
                <h2 style="color: #800020;">Página {page_number}</h2>
                <div style="background-color: #f0f0f0; padding: 10px; margin-bottom: 20px;">
                    <h3>Texto Original (Página {page_number})</h3>
                    <p>{formatted_text}</p>
                    <h3>Texto Traduzido (Página {page_number})</h3>
                    <p>{translated_page}</p>
                </div><br>
                """)
                container.children += (page_display,)

                # Criar o botão de gerar áudio
                audio_button = Button(description=f'Gerar Áudio da Página {page_number}')
                audio_button.on_click(lambda b, t=page_text, s=text_snippet: generate_audio(b, t, s))
                container.children += (audio_button,)

        # Exibir o conteúdo do container
        display(container)

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo: {e}")
        display(HTML(f"Erro ao processar o arquivo: {str(e)}"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)